In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("ETL"))))

In [2]:
from ETL import T


함수에 대한 설명은 아래와 같습니다. 

라이브러리는 ETL과정중 T(Transform)을 포함하고 있습니다. 

데이터베이스에서 데이터를 확인하는 클래스는 Checker 입니다. 

클래스 지정시 데이터베이스(postgresql)의 id, pw, ip, db, 추출하고자 하는 테이블명을 입력해주세요. 

read_excel() 함수는 xlsx을 불러와 데이터프레임으로 저장합니다. 

read_csv() 함수는 csv를 불러와 데이터프레임으로 저장합니다. 

data_update() 함수는 신규 데이터의 업데이트시 I 또는 U를 입력합니다. 

date_update() 함수는 신규 데이터의 업데이트시 날짜를 입력합니다. 

CheckDate() 함수는 인베스팅 닷컴의 일반 데이터 날짜를 표준화하는 함수 

CheckLength() 함수는 데이터의 크기를 확인하여 크기만큼 자르는 함수 

CheckVarchar() 함수는 재무 데이터 크기를 확인하여 클 경우 새로 삽입하는 함수 

CheckNumeric() 함수는 재무 데이터 숫자를 확인하는 함수 

------------------------------------------------------------------ 

데이터베이스에서 데이터를 확인하는 클래스는 Analysis 입니다. 

read_excel() 함수는 xlsx을 불러와 데이터프레임으로 저장합니다. 

read_csv() 함수는 csv를 불러와 데이터프레임으로 저장합니다. 

Fail() 함수는 오류가 난 데이터를 데이터프레임에 넣기 위해 딕셔너리화 시키는 함수 

CheckDate_Duplicate() 함수는 날짜체크, 중복체크를 확인하는 함수 

CheckNumber() 함수는 전화번호가 유효한지 확인하는 함수 



In [8]:
path = "fn/fn_unfinished/"

In [9]:
filepath = os.listdir(path)

In [10]:
tmp = T.Analysis()

In [11]:
for idx in filepath:
        try:
            t = T.Checker()
            t.read_excel(path + idx)
            t.df = tmp.change_columns(t.df)
            t.CheckLength()
            t.CheckNumeric()
            t.CheckVarchar()
            try: t.CheckDate()
            except: pass
            t.data_update(True)
            t.date_update()
            t.fndtn_dt()
            t.df.to_excel(f"fn/unload/{idx}", index=False)
        except:
            t = T.Checker()
            t.read_excel(path + idx)
            t.CheckLength()
            t.CheckNumeric()
            t.CheckVarchar()
            t.data_update(True)
            t.date_update()
            t.fndtn_dt()
            t.df.to_excel(f"fn/unload/{idx}", index=False)


c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:
c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:
c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:
c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:
c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or 

In [11]:
t.df["entrp_reltn_tdngs_dt"].isnull().value_counts()

True    796
Name: entrp_reltn_tdngs_dt, dtype: int64

In [12]:
import os
path = "fn/unload/"
filepath = os.listdir(path)

In [13]:
filepath

['finished_australia.xlsx',
 'finished_colombia.xlsx',
 'finished_france.xlsx',
 'finished_germany.xlsx',
 'finished_hong-kong.xlsx',
 'finished_indonesia.xlsx',
 'finished_italy.xlsx',
 'finished_malaysia.xlsx',
 'finished_mexico.xlsx',
 'finished_netherlands.xlsx',
 'finished_singapore.xlsx',
 'finished_spain.xlsx',
 'finished_thailand.xlsx',
 'finished_united-states.xlsx',
 'finished_vietnam.xlsx',
 'Original_미국_2000.xlsx',
 'Original_미국_4000.xlsx',
 'Original_미국_6000.xlsx',
 'Original_스위스_834.xlsx',
 'Original_영국_2000.xlsx',
 'Original_영국_2624.xlsx',
 'Original_인도_2000.xlsx',
 'Original_인도_2090.xlsx',
 'Original_캐나다_1846.xlsx']

In [14]:
from ETL import EL

In [15]:
tt = EL.DataLoad()

In [16]:
cnt = 0
for idx in filepath:
    try:
        os.rename(path+idx, "fn/unload/" + tt.change_name(idx))
    except FileExistsError:
        cnt += 1
        os.rename(path + idx, "fn/unload/" + str(cnt) + tt.change_name(idx))

In [17]:
import pandas as pd
df = pd.read_excel("fn/fn_unfinished/Original_미국_2000.xlsx")

In [19]:
t.read_excel("fn/fn_unfinished/Original_미국_2000.xlsx")

t.CheckLength()
t.CheckNumeric()
t.CheckVarchar()
t.data_update(True)
t.date_update()
t.fndtn_dt()

In [32]:
t.fndtn_dt()

c:\Users\user\Desktop\yoo\개발관련\BDC\Digital-Industry-Innovation\DataQuality\ETL\T.py:140: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:


In [35]:
t.df[t.df["accnn_yr"] >= 2022]

,keyval,stock_mrkt_cd,acplc_lngg_stock_mrkt_nm,engls_stock_mrkt_nm,hb_ntn_cd,lstng_cd,acplc_lngg_entrp_nm,engls_entrp_nm,ovrss_entrp_crprt_rgno,ovrss_entrp_bsnsm_rgno,...,entrp_reltn_tdngs_url,cmp_insd_reltn_info,cstmr_reltn_info,sppl_reltn_info,cmptt_reltn_info,sbst_goods_reltn_info,opert_sctin_cd,data_crtin_dt,cntct_prces_stts_cd,cntct_prces_dt
0,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",NaN,77-0518772,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
1,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",NaN,77-0518772,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
2,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",NaN,77-0518772,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
3,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",NaN,77-0518772,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
4,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",NaN,77-0518772,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5989,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,APRN,"Blue Apron Holdings, Inc.","Blue Apron Holdings, Inc.",NaN,81-4777373,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
5990,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,APRN,"Blue Apron Holdings, Inc.","Blue Apron Holdings, Inc.",NaN,81-4777373,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
5991,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,APRN,"Blue Apron Holdings, Inc.","Blue Apron Holdings, Inc.",NaN,81-4777373,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN
5992,NaN,NYSE,New York Stock Exchange,New York Stock Exchange,NaN,APRN,"Blue Apron Holdings, Inc.","Blue Apron Holdings, Inc.",NaN,81-4777373,...,NaN,NaN,NaN,NaN,NaN,NaN,I,20230426,n,NaN


In [38]:
empty_data = {
    "aus" : ["Australian Securities Exchange", "ASX"],
    "can" : ["Toronto Stock Exchange", "TSX"],
    "che" : ["SIX Swiss Exchange", "SIX"],
    "col" : ["Bolsa de Valores de Colombia", "BVC"],
    "deu" : ["Frankfurt Stock Exchange", "FSE"],
    "esp" : ["Bolsa de Madrid", "BM"],
    "fra" : ["Euronext Paris", "EP"],
    "gbr" : ["London Stock Exchange", "LSE"],
    "hkg" : ["Hong Kong Stock Exchange", "HKEX"],
    "idn" : ["Indonesia Stock Exchange", "IDX"],
    "ind" : ["National Stock Exchange", "NSE"],
    "ita" : ["Borsa Italiana", "BI"],
    "jpn" : ["Tokyo Stock Exchange", "TSE"],
    "kor" : ["Korea Exchange", "KRX"],
    "mys" : ["Kuala Lumpur Stock Exchange", "KLSE"],
    "mex" : ["Bolsa Mexicana de Valores", "BMV"],
    "nld" : ["Euronext Amsterdam", "BA"],
    "sgp" : ["Singapore Stock Exchange", "SGX"],
    "tha" : ["Stock Exchange of Thailand", "SET"],
    "usa" : ["New York Stock Exchange", "NYSE"],
    "vnm" : ["Ho Chi Minh City Stock Exchange", "HSX"]}

In [39]:
country_name = "tb_hb_aus_plcfi_d"

In [42]:
path = "fn/unload/"
filepath = os.listdir(path)

ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Securities Exchange Australian Securities Exchange aus
ASX Australian Secur